In [ ]:
!pip install -U openai-whisper
!pip install srt
!pip install openai

In [ ]:
import subprocess
import openai
import whisper
from datetime import timedelta
import srt
from srt import Subtitle


# 音声データ抽出 (音声ファイル名変更)
subprocess.run(['ffmpeg', '-i', 'test.mp4', '-acodec', 'libmp3lame', '-ab', '256k', 'audio.mp3'])

# Whisperモデル読み込み
model = whisper.load_model("large-v3")

# 音声ファイル読み込み
audio = whisper.load_audio("audio.mp3")

# 音声認識 (開始・終了時間付きセグメント取得)
result = model.transcribe(audio, verbose=True)
segments = result["segments"]

# SRTファイル用の空リスト初期化
subtitles = []

# セグメントごとにSRT行を作成
for index, segment in enumerate(segments, start=1):
    start_time = timedelta(seconds=segment["start"])
    end_time = timedelta(seconds=segment["end"])
    text = segment["text"]

    subtitle = Subtitle(
        index=index,
        start=start_time,
        end=end_time,
        content=text
    )
    subtitles.append(subtitle)

# SRTファイル内容の作成
srt_content = srt.compose(subtitles)

# ファイルへの書き込み (エンコーディング指定)
with open("output.srt", "w", encoding="utf-8") as f:
    f.write(srt_content)

# SRTファイル読み込み (エンコーディング指定)
with open('output.srt', 'r', encoding='utf-8') as file:
    srt_data = file.read()

# OpenAI APIキー設定
openai.api_key  = 'your_api_key'

# 翻訳リクエスト
completions = openai.chat.completions.create(
    model="gpt-4-0125-preview",
    messages=[
        {"role": "user", "content": f"[SRT形式のまま日本語に翻訳してください]:\n{srt_data}"}
    ],
)

srt_text = completions.choices[0].message.content

# 改行関数 (SRT形式保持)
def add_line_breaks_srt(srt_text):
    lines = srt_text.split("\n")
    result = ""

    for i, line in enumerate(lines):
        if line.strip().isdigit():
            result += line
        elif "-->" in line:
            result += line
        else:
            result += add_line_breaks(line) + "\n"

        # 空行を追加 (字幕間の空白)
        if i < len(lines) - 1 and lines[i + 1] != "" and line != "":
            result += "\n"

    return result.strip()

# 改行関数 (句読点・句点での改行)
def add_line_breaks(text):
    result = ""
    line_length = 0

    for char in text:
        if char == "、" and line_length >= 7:
            result += "\n"
            line_length = 0
        elif char == "。":
            result += "\n"
            line_length = 0
        else:
            result += char
            line_length += 1

    return result

# 改行されたテキストを取得
result = add_line_breaks_srt(srt_text)

# ファイルへの書き込み (エンコーディング指定)
with open("output.srt", "w", encoding='utf-8') as file:
    file.write(result)

print(result)
print("ファイルに書き込みました。")

# 字幕動画生成 (縦動画)
subprocess.run(['ffmpeg', '-i', 'test.mp4', '-vf', "subtitles=output.srt:force_style='FontName=Helvetica,FontSize=6'", 'test2_subtitle.mp4'])